# Generator based Coroutine

## basic behavior

In [1]:
def simple_coroutine(): 
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()

In [3]:
my_coro

<generator object simple_coroutine at 0x7f18f9138a00>

In [4]:
type(my_coro)

generator

In [5]:
next(my_coro)

-> coroutine started


In [6]:
my_coro.send(30)

-> coroutine received: 30


StopIteration: 

## Coroutine (actually generator) Status (Life cycle)

In [7]:
def simple_coro2(a): 
    print('-> Started: a =', a)
    b= yield a
    print('-> Received: b =', b)
    c = yield a+b
    print('-> Received: c =', c)

In [8]:
my_coro2 = simple_coro2(14)

In [9]:
from inspect import getgeneratorstate

In [10]:
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [11]:
my_coro2.send(12)

TypeError: can't send non-None value to a just-started generator

In [12]:
next(my_coro2)

-> Started: a = 14


14

In [13]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [14]:
my_coro2.send(50)

-> Received: b = 50


64

In [15]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [16]:
my_coro2.send(1050)

-> Received: c = 1050


StopIteration: 

In [17]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

In [18]:
my_coro2.send(40)

StopIteration: 

In [19]:
next(my_coro2)

StopIteration: 

In [20]:
my_coro3 = simple_coro2(50)

* 코루틴 동작 방식 (그림 출처 : fluent python(2015))

![gen_coro_flow](./images/gen_coro_flow.png)

## Sample : 이동평균을 계산하는 코루틴

In [21]:
def averager():
    total = 0.0
    count = 0
    average = None 
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count


In [22]:
coro_avg = averager()
next(coro_avg)

In [23]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [24]:
coro_avg.send(30)

30.0

In [25]:
coro_avg.send(40)

35.0

In [26]:
coro_avg.send(None)

TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'

In [27]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

## 코루틴의 종료와 예외처리

In [28]:
coro_avg = averager()
next(coro_avg)
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [29]:
coro_avg.send(50)

50.0

In [30]:
coro_avg.send(120)

85.0

In [31]:
coro_avg.close()

In [32]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

### 코루틴 예외처리를 방법을 설명하기 위한 제너레이터

In [33]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling(): 
    print('-> coroutine started') 
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [34]:
exc_coro = demo_exc_handling()
next(exc_coro)
getgeneratorstate(exc_coro)

-> coroutine started


'GEN_SUSPENDED'

In [35]:
exc_coro.send(11)

-> coroutine received: 11


In [36]:
exc_coro.send(20)

-> coroutine received: 20


In [37]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [39]:
exc_coro.throw(Exception)

Exception: 

In [40]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [41]:
exc_coro.close()

In [42]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

#### 예외 발생시키기

In [44]:
exc_coro = demo_exc_handling()
next(exc_coro)
getgeneratorstate(exc_coro)

-> coroutine started


'GEN_SUSPENDED'

In [45]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [46]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [47]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [48]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

#### 코루틴 종료시 어떤작업을 실행하려면 try-finally 구문 사용

In [49]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_finally():
    print('-> coroutine started') 
    try:
        while True: 
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...') 
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [50]:
exc_coro = demo_finally()
next(exc_coro)
getgeneratorstate(exc_coro)

-> coroutine started


'GEN_SUSPENDED'

In [51]:
exc_coro.send(11)

-> coroutine received: 11


In [52]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [53]:
exc_coro.throw(ZeroDivisionError)

-> coroutine ending


ZeroDivisionError: 

## 코루틴에서 값을 반환하기

In [55]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None 
    while True:
        term = yield average 
        if term is None:
            break
        
        total += term
        count += 1
        average = total/count
    return Result(count, average)


In [56]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [57]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
print(result)

Result(count=3, average=15.5)


In [58]:
coro_avg = averager()
next(coro_avg)

In [59]:
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

15.5

In [60]:
r = coro_avg.close()

In [61]:
print(r)

None


## yield from

In [62]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i


In [63]:
gen()

<generator object gen at 0x7f18f019d9a0>

In [64]:
list(gen())

['A', 'B', 1, 2]

In [65]:
def gen():
    yield from 'AB'
    yield from range(1,3)


In [66]:
gen()

<generator object gen at 0x7f18f03c75e0>

In [67]:
list(gen())

['A', 'B', 1, 2]

In [71]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
        
    def __repr__(self):
        return f"Senetence ({reprlib.repr(self.text)})"

In [72]:
def sentence_gen():
    yield from iter(Sentence("The old man and sea"))
    yield from Sentence("Crime and Punishment")

In [73]:
sentence_gen()

<generator object sentence_gen at 0x7f18f03c7370>

In [74]:
list(sentence_gen())

['The', 'old', 'man', 'and', 'sea', 'Crime', 'and', 'Punishment']

### generator와 함께 사용

In [76]:
def main_gen():
    while True:
        yield from averager()

In [77]:
main = main_gen()

In [78]:
main

<generator object main_gen at 0x7f18f01a5b40>

In [79]:
next(main)

In [80]:
main.send(30)

30.0

In [81]:
main.send(4)

17.0

In [82]:
main.send(60)

31.333333333333332

In [ ]:
r = main.send(None)

In [ ]:
def main_gen2():
    while True:
        yield from averager()
        yield from sentence_gen()

In [ ]:
main = main_gen2()
next(main)

In [ ]:
main.send(30)

In [ ]:
next(main)

In [ ]:
next(main)

In [ ]:
main.send(40)

In [ ]:
main.send(40)

In [ ]:
next(main)

In [ ]:
list(main)

### Sample : 가상의 중1 학생 키와 몸무게 평균

In [86]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
# the subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


# the delegating generator
def grouper(results, key): 
    while True:
        results[key] = yield from averager()

# the client code, a.k.a. the caller
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key)
        print(f"{key}, {type(group)}, {id(group)}")
        next(group)
        for value in values:
            group.send(value) 
        #group.send(None) # important!
        print(results)  # uncomment to debug
    report(results) 
        
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count:2} {group:5} averaging {result.average:.2f}{unit}')

data={ 
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

girls;kg, <class 'generator'>, 139744727834608
{}
girls;m, <class 'generator'>, 139744727835024
{}
boys;kg, <class 'generator'>, 139744727834608
{}
boys;m, <class 'generator'>, 139744727835024
{}


In [84]:
data={ 
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}
for k , v in data.items():
    print(f"{k = },{v = }")

k = 'girls;kg',v = [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5]
k = 'girls;m',v = [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43]
k = 'boys;kg',v = [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3]
k = 'boys;m',v = [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]


![averager](./images/group_averager.png)

In [ ]:
data.items()

In [ ]:
for k, v in data.items():
    print(f"{k = }, { v =}")

# Async & Await 

In [87]:
def hello():
    print("Hello, Old-Fashioned World!")

hello()

Hello, Old-Fashioned World!


In [88]:
async def async_hello():
    print("Hello, Asynchronous World!")

async_hello()

<coroutine object async_hello at 0x7f18f01a5c00>

In [89]:
def gen_hello():
    yield 
    print("Hello, Generator World!")

gen_hello()

<generator object gen_hello at 0x7f18f01a5e40>

In [90]:
coro_hello = async_hello()
#next(coro_hello)
try:
    coro_hello.send(None)
except StopIteration as ex:
    print(ex.value)
    

Hello, Asynchronous World!
None


In [102]:
import types

@types.coroutine
def lazy_range(n):
    while n >= 0:
        yield n
        n -= 1
    return n
    
async def print_list():
    res = await lazy_range(5)
    print(f"{res = }")

In [103]:
coro_list = print_list()
try:
    coro_list.send(None)
    while True:
        print("Step", coro_list.send(None))
except StopIteration as ex:
    print(ex.value)

Step 4
Step 3
Step 2
Step 1
Step 0
res = -1
None


In [104]:
a = lazy_range(5)
a

<generator object lazy_range at 0x7f18f01a6980>

### Sentence를 Awaitable로 구현하기

In [91]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text

    def __await__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
        
    def __repr__(self):
        return f"Senetence ({reprlib.repr(self.text)})"

In [92]:
async def get_word():
    await Sentence("the old man and sea")

In [93]:
get_word

<function __main__.get_word()>

In [94]:
word = get_word()
word

<coroutine object get_word at 0x7f18f01a5f00>

In [95]:
try:
    while True:
        print("Step", word.send(None))
except StopIteration as ex:
    print(ex.value)

Step the
Step old
Step man
Step and
Step sea
None


In [96]:
word.send(None)

RuntimeError: cannot reuse already awaited coroutine

### Averager를 awitable로 구현하기

In [105]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

#import types

@types.coroutine
# the subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


In [106]:
await_avg = averager()
await_avg

<generator object averager at 0x7f18da73b2e0>

In [107]:
await_avg.send(None) # start

In [108]:
await_avg.send(30)
await_avg.send(40)
await_avg.send(50)

In [109]:
await_avg.send(None)

StopIteration: Result(count=3, average=40.0)

In [110]:
# the delegating generator
async def grouper(results, key): 
    while True:
        #results[key] = yield from averager()
        results[key] = await averager()

# the client code, a.k.a. the caller
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key)
        print(f"{key}, {type(group)}, {id(group)}")
        #next(group)
        group.send(None)
        for value in values:
            group.send(value) 
        group.send(None) # important!
        # print(results)  # uncomment to debug
    report(results) 
        
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count:2} {group:5} averaging {result.average:.2f}{unit}')

data={ 
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

girls;kg, <class 'coroutine'>, 139744727835232
girls;m, <class 'coroutine'>, 139744724252336
boys;kg, <class 'coroutine'>, 139744727835232
boys;m, <class 'coroutine'>, 139744724252336
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [114]:
results = {}
key = ""
group = grouper(results, key)
group

<coroutine object grouper at 0x7f18dadc85f0>

In [ ]:
a.send(None)

In [ ]:
a.send(10)
a.send(20)
a.send(30)
a.send(40)

In [ ]:
a.send(None)